# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [2]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [3]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome(executable_path='./chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    time.sleep(SCROLL_PAUSE_TIME)
    
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面最底端")
        break
    last_height = new_height
    print('網頁更新中...')

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面最底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [6]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 2萬元就一次包下「18支金融股」　這檔ETF今年漲破20%！

▲投資人愛存金融股，不知該存哪一檔時可留意金融ETF。（資料照）
圖文／鏡週刊
本週起，國內多家金融股股東會密集召開，由於獲利普遍成長，今年金融股配息金額突破2,157億元，其中更有7家金控公司配息金額超過百億。近期外資積極回補金融股，儼然成為動盪局勢下令人放心的安全牌。
上週五（16日）台股上市金融類股指數創下1997年底來新高，國內唯一一檔「純金」ETF--元大MSCI金融（0055），也同步寫下歷史新高；但20日受美股重挫衝擊，下跌1.57%、收盤價22.01元。
進一步來看，元大MSCI金融近3年配息率都在3.5%以上，受到新冠肺炎疫情影響，全球持續低利率環境，金融股表現相對落後；今年則在通膨隱憂升溫、利率政策調整預期下，金融股開始落後補漲，元大MSCI金融ETF股價一路穩步向上，站穩20元關卡，並於日前寫下歷史新高22.36元。
元大MSCI金融ETF每年約10月底進行收益評價，11月左右除息，投資人若看好台股多頭續航力，想參與金融股配息，卻又懶得自己追蹤個別金融股基本面，可考慮透過布局純金ETF卡位。
元大投信表示，觀察金融股發放金額最高前幾名，包括國泰金、富邦金、兆豐金，及市場上熱門存股名單，像是中信金、玉山金、開發金等，一檔元大金融ETF全部涵蓋。投資人不用再猶豫該存哪一檔好，只要投入2萬元左右，就可打包一籃子金融股。
----------------------------------------------------------------------

[1] 淹水可理賠！烟花來襲　金管會籲及早投保「颱風洪水險」

▲烟花來襲，金管會提醒民眾應做好防颱準備及注意自身安全。（圖／記者屠惠剛攝）
記者陳依旻／台北報導
烟花來襲！金管會提醒民眾應做好防颱準備及注意自身安全，及早投保颱風洪水保險，如有淹水損失，可在災害發生後申請理賠金。
金管會表示，民眾投保之保單如有包含因颱風或洪水所致之損失者，如住宅火災保險、火災保險附加「颱風及洪水保險」、「地層下陷、滑動或山崩保險」、汽車車體損失保險附加「颱風、地震、海嘯、冰雹、洪水或因雨積水附加條

▲全國自27日起降為二級警戒，百貨賣場、傳統市場和夜市、餐廳等場所可開放內用。（圖／記者李毓康攝）
記者林淑慧／台北報導
行政院拍板27日起調降為二級警戒，並開放餐飲內用與婚宴，不過，台北市長柯文哲、新北市長侯友宜均宣佈餐飲內用解禁擺最後。對此，經濟部政務次長陳正祺表示，指揮中心放寬餐飲業內用措施出爐，可望讓業者有喘息機會，希望地方政府在防疫之餘，也能兼顧民生經濟發展需求；另外，將待國內疫情獲得控制並趨緩後，規劃相關振興措施，定案後將由行政院正式公佈。
依據經濟部統計，6月餐飲業營業額年減39.9%，為有統計以來最大減幅，其中佔餐飲業大宗的餐館業年減40.4%；陳正祺表示，餐飲業因禁止內用受到衝擊最深，營收表現不樂觀，中央宣佈降為二級警戒、開放餐飲內用，也是希望讓餐飲業能夠有喘息的機會，兼顧民生經濟發展。
近期疫情已漸漸趨緩，往好的方向發展，中央疫情指揮中心今日宣布可有條件放寬餐飲內用原則，依衛福部規定婚宴、公祭可開放，經濟部商業司長蘇文玲指出，婚宴開放宴客，餐飲比照餐飲業指引，婚宴仍必須符合室內五十人、室外一百人以下的規範，婚宴主婚人、新郎及新娘可在台上向賓客敬酒，並遵守餐飲指引不得逐桌敬酒。
針對美容美體業等有近距離接觸的業別，於三級警戒期間不開放，經與業者溝通後，降為二級時業者可開放採預約制營業，臉部美容等須拿下口罩的營業項目先不開放，而執行業務的空間希望避免密閉，如為包廂門須半開，每位客人須間隔十五分鐘以上 。
至於娃娃機、自動選物販賣機等業別，蘇文玲說，很多業者反映能在營業空間，做到自主管理，因此同意有條件開放；要求業者針對環境定期消毒，每一機台必須消毒完才能給下一組客人使用，要求必須有專人管理人流及清消工作。
----------------------------------------------------------------------

[9] 富邦設置助威金「每人最高280萬」　力挺選手征戰奧運！

▲富邦設置助威金「每人最高10萬」，力挺選手征戰奧運。（圖／富邦集團提供）
記者陳依旻／台北報導
萬眾矚目的東京奧運正式開幕，為鼓舞士氣、預祝選手能順利奪金，富邦集團今（23）日宣布，將設置金牌10萬美元、銀牌5萬美元、銅牌2.5萬美元的富邦助威金，為富邦金控及台灣大哥大贊助的選手郭婞淳、潘政琮、楊勇緯、徐薇淩、李旻、詹詠然、詹皓晴等集團應援

▲金管會因應降級，宣布股東會可比照第2級，室內不得超過50人、室外不可超過100人。（示意圖／取自免費圖庫Pixabay）
記者陳依旻／台北報導
全國疫情趨緩，全台3級警戒確定如期調降至2級，股東會人數限制隨之調整，金管會今（23）日表示，放寬室內人數可50人、室外100人以內。
原本上市櫃公司每年都要在6月底前舉行股東會，但今年因疫情緣故，首度延後至7月1日至8月底召開，原本規定室內不得超過5人，室外不得10人，但讓大多數公司在籌備上有實務困難，所以，金管會與指揮中心呈報後並獲准在兼顧防疫需求之下，放寬為各室內不得20人以上、各室外不得40人以上。
因應政府今日宣布降級，證期局官員今日宣布，遵照指揮中心最新公告辦理，放寬室內人數可50人、室外可在100人以內，也會督導集保公司修正防疫作業指引以利公開發行公司順利召開。
證期局官員也呼籲各公司、股務機構遵守，股東會全程戴口罩、實名制、禁止飲食以及量測體溫、發燒不得進入會場、座位採梅花座間隔坐法。
----------------------------------------------------------------------

[18] 快訊／ATM及網銀手續費恢復原收費！　8月3日0時生效　

▲因應降級，ATM及網銀手續費8月3日凌晨0時起恢復原收費。（示意圖／取自免費圖庫Pixabay）
記者陳依旻／台北報導
全國疫情趨緩，全台3級警戒確定如期調降至2級，手續費持續全部減免，實體ATM轉帳手續費減收2元不再，金管會銀行局今（23）日宣布，ATM及網銀手續費收費8月3日凌晨零時起恢復原標準。
在疫情3級警戒期間，金管會提出暫行措施，其中，有3大重點。第一， ATM及網銀手續費優惠；第二， 保險業務員以「視訊」取代親晤親簽；第三， 股東會舉行人數室內不得20人、室外不得40人以上。
行政院今公布自7月27日起，將3級警戒調降至2級警戒，讓符合防疫指引的各行業、場所能夠恢復活動。
金管會銀行局日前在全國3級警戒期間，宣布各銀行網路銀行與行動銀行的跨行轉帳包括繳稅費在內，手續費持續全部減免，實體ATM轉帳手續費減收2元，
金管會銀行局今日表示，配合中央流行疫情指揮中心宣布自110年7月27日起疫情警戒調降至第2級，有關3級警戒期間所發布之銀行業相關因應措施，說明如下：
三、有關銀行業以彈性方式提供金融商品及服

▲台驊董事長顏益財相當看好貨櫃船運市場前景。(圖／記者張佩芬攝)
記者張佩芬／台北報導　
台驊(2636)因為一年內累積取得同一有價證券達實收資本額20％，昨(23)晚近9點公布今年1月18日到7月22日加碼進陽明(2609)4,900張，平均價格60.3元，以今日陽明收盤價145.5元估算，可獲利金額達4.17億元，投資報酬率達141.36％。
顏益財之前曾表示，公司今年農曆年前及本土疫情爆發前持續加碼陽明，今年以來累計投入2.95億元買進陽明4,900張股票。台驊原參加私募的1萬張買進價格是10.48元，參加現增的5千張買進價格是12元。
以該公司公告目前共有持有陽明股票1.91餘萬張，顯示是有賣出約800張。公司高階指出，是有小出狀況，但買得更多，董事長顏益財所說的，「一張不賣，反向加碼」，是看好陽明、信任陽明，小賣不是不看好，是逢高時賣出一點，股價被投資人低估時多買進一些。台驊目前持有的陽明股份佔該公司總股數0.57％。
台驊本月15日因陽明股價低於增資股定價，宣布放棄參與陽明此次現金增資，但顏益財以個人名義參與認購，仍持續看好未來貨櫃船運市場。
----------------------------------------------------------------------

[29] 泰豐處分中壢廠　大股東南港輪胎急發函要求「開股臨會決定」

▲泰豐經營權之爭仍繼續，圖為日前泰豐最大股東南港輪胎委任的法人董事長趙國帥前往台北地檢署按鈴申告，控泰豐輪胎公司董事長馬述健。（資料圖／記者劉昌松攝）
記者吳佳穎／台北報導
針對泰豐輪胎公司董事會決議處分中壢廠，最大股東南港輪胎今（23）日主張依公司法規定，處分主要部分的財產應由全體股東以特別決議同意，不可由董事會決議即輕率執行，以免戕害全體股東權益。今下午緊急發函泰豐董事會，要求立即召開股東臨時會，由全體股東共同討論表決處分案。
南港輪胎董事長張昌平發函指出，泰豐輪胎刻意忽視公司法第185條相關規定之違法行徑，剝奪一般股東合理預期應可依法定程序慎重研討重大處分案，更與主管機關嚴予要求的公司治理原則背道而馳，應受嚴厲譴責，並立即制止。泰豐董事會若執意違法，也應負民刑事責任。
由於泰豐輪胎7月22日公告，董事會決議將以底價新台幣43.94億元、56.5億元，分別就泰豐100%持有之子公司泰鑫建設全部股權及子

▲原PO把錢輸光了。（示意圖／取自Pakutaso）
記者曾筠淇／綜合報導
許多人都會透過投資股票來增加自身被動收入，就有一名男網友發文表示，他先前因在股市嘗到甜頭，所以便融資開槓桿約2000萬元，怎料最後投資失利，全部輸光光。貼文曝光後，立刻引發熱議，不過也有些網友直言，「現在大學生主修幻想了」、「反串吧」。
原PO透露，現在雖然成功畢業，但是就算拿出所有存款，依舊賠不完在股市中慘虧的錢，「我該怎麼辦？」而這樣的情況，也讓他不敢向家人求助，只能獨自哭泣。
▲原PO不知所措。（圖／翻攝自Dcard）
貼文曝光後，網友紛紛留言回應，「錢再賺就好，但命只有一條」、「36萬元翻到快1700萬元很猛」、「跟家人求救一下，頂多被罵個幾句，不會見死不救」、「沒事啦～好好工作吧」、「沒事，扣完才負300多萬元，OK的！錢再賺就有了」、「有好幾個股神都是破產後再出發的」。
不過，也有不少網友質疑事件的真實性，「如果是反串那還真爛」、「我怎麼覺得在反串」、「你哪裡抓的圖」、「實在不相信這種心情狀態還會有興致到Dcard上討拍」、「我覺得要先把負債的圖片放上來才有公信力」、「請問你拿什麼開槓桿借到2000萬元」、「反串吧？這一波又沒有跌停鎖死，融資頂多被斷頭，又不是賣都賣不掉」、「坐等對帳單」。
----------------------------------------------------------------------

[40] 老牌上市建商估房市將「價穩量盤整」　全台推案總銷衝190億

▲國建與日商三井不動產合作的「UNi PARK」目前已銷售8成。（圖／記者陳建宇攝）
記者陳建宇／綜合報導
國泰建設（2501）今（23）日舉行年度股東會，該公司去年營收達133.36億元，今年將配發1元的現金股利，看好下半年房市發展，將推出總銷共190億元個案，該公司也積極在全台布局，今年購入的4筆土地遍布北中南，更與三井不動產合作2案，朝綜合開發商發展。
►《地產詹哥老實說》S2EP22／東區10年起落！店租打5折降盼留租客　傳奇房東沒在怕？
國建今（23）日舉行年度股東會，該公司指出，2020年全年營收達到133.36億元，每股盈餘為1.28元，會中並決議發放現金股利1元。而針對今年下半年的房市，國建認為疫情趨緩之後，許多延後推出的個案將會大舉進場，因此預估市場供給量將大增，

▲宅家生活，全民防疫瘋網購。（圖／shutterstock）
記者陳依旻／台北報導
疫情高低起伏，配合政府防疫措施，民眾遵守宅家防疫措施，減少非必要外出，大大改變的許多人的日常生活。利用網購解決一日三餐或是生活所需變成常態，在家時間長了，沒事點開購物網站消費，也是宅家生活紓壓的一種方式，大家最想知道的就是刷哪些卡可以得到最多的回饋？趕緊來瞭解一下，網友討論熱烈的2021優惠神卡：電商神卡英雄聯盟信用卡、中信Line Pay信用卡。
▲中信英雄聯盟信用卡，超強現金回饋10%大敗眾多對手。（圖／中國信託提供）
說到網購，不得不推薦口碑最強勢的英雄聯盟信用卡，光是七大指定電商像是PChome線上購物 、MOMO購物網、Yahoo購物中心、蝦皮購物、淘寶網、東森購物、ViVa TV大家常使用的平台，在年底前都有最高10%現金回饋就贏過眾多對手，其他包含民眾一日生活使用最頻繁的Uber Eats、foodpanda，宅在家裡追劇必備的Netflix、KKBOX、LINE TV等，若要外出減少大眾交通系統搭乘Uber 同樣也有最高10%優惠，新申辦、推薦辦卡也有額外好禮相贈。每日生活消費刷這張回饋最多。
▲民眾宅在家除了看劇、打遊戲，網購也是一種樂趣。（圖／shutterstock）
2016年推出LINE Pay聯名卡。靠著持續擴大LINE POINTS生態圈經營與不斷精進的數位體驗互動，一直都是信用卡市場上明星商品。台灣信用卡歷史很難有發行超過200萬張的卡片，今年７月中信LINE Pay卡已來到400萬張總發行量，成為台灣發卡量最高的聯名卡。慶祝400萬卡將祭出更多優惠和募卡禮，8月份申辦且於核卡30日內，首刷滿NT888元送100元刷卡金加碼禮與專屬新戶禮LINE POINTS 500點等好禮，新卡友消費滿NT3,888元，再送LINE Friends授權20吋行李箱乙個(限量4,600個)。8月底前指定通路最高享6%回饋，像是MOMO、屈臣氏、博客來、中油、台亞、全國加油站、還有Foodpanda、王品集團等通路都可享滿額加碼回饋。此外，至12月底前，LINE Pay卡以『天天回饋無上限』、『處處優惠賺很大』、『筆筆條碼支付便』、『時時網購賺很大』為優惠主軸，包辦你的一日生態圈，精選數十家商店消費加碼，最高可享LINE POINTS 15%回饋。若是要出門，搭配LIN

▲和潤、和運董事長由劉源森接任，期待透過新業務開發，再創事業高峰。（圖／和泰提供）
記者高兆麟／綜合報導
和泰集團旗下和潤企業及和運租車董事會改選新任董事長，由劉源森當選兩家公司董事長一職。新任董事長劉源森任職於和泰集團已達37年，自1984年加入和泰汽車，於豐田車進入台灣巿場時即參與新事業籌備規劃，在豐田車業務部門歷練。由於擅於經營策略規劃及改善改革，後續派至國都汽車擔任總經理，之後集團購入蘇黎世產險轉任和泰產險副董事長，在兩家公司期間，無論營收或是獲利均能逐年創造新高紀錄。
劉源森接任新職時表示：和潤、和運兩家公司成立23年，在董事長田天明及經營團隊領導之下，已經成為台灣最大的汽車分期與租賃公司，尤其近三年來更維持兩位數以上的經營成長。未來我們將努力保持此一成長氣勢，並秉持集團蘇總裁「没有最好、只有更好」的理念，期望透過擴大新業務項目、新地域市場的開發，和活用新IT進行管理的優化，並加速展開各種人才培育養成，以締造和潤3.0、和運3.0(第三個十年)之新局，再創兩家公司的另一事業高峰。
----------------------------------------------------------------------

[61] 快訊／8成疫苗被退貨？　高端澄清：數字全然錯誤

▲高端針對被退貨一事發聲明澄清。（圖／記者湯興漢攝）
記者廖婕妤／綜合報導
高端疫苗（6547）今早被爆出在擴大產能的情況下，由於數據偏差太多，導致有82％被退貨，影響到今日早盤開低走低，大約上午10點打入跌停，股價來到254.5元。不過，高端對於被退貨一事出面澄清，指出抗原產能遠高於媒體報導臆測之數值，所有疫苗成品皆會100%符合規格，並在完成廠內放行與食藥署研檢封緘程序後釋出，媒體臆測有關生產良率及每批產能等數字全然錯誤。
高端疫苗針對疫苗遭退貨一事聲明澄清，指出報導內容陳述50L產能規模只有8.8萬劑、82%遭退貨品質堪憂、抗原品質偏差大、200L製程放大全軍覆沒等資訊，係屬不實資訊，請媒體勿擷取不明來源之資訊進行不實報導。
高端表示，疫苗從關鍵原物料生產、到檢測放行，不但規格嚴謹、項目繁多、且耗時長。從最前端的入料檢驗、製程中管控、到最終端成品放行，須完成超過百項嚴謹檢驗，始可放行。
高端說明，目前本公司抗原產能遠高於媒體報導臆測之數值，抗原庫存量也大於目前國內採購規劃之

記者吳珍儀／綜合報導
美國經濟數據雖好壞參半，但企業發布財報大多正向，美股連續第3天收漲。台股今（23）日以17603點開出，上漲31點。
蘋概三王中，台積電（2330）今日以592元開出，上漲1元；大立光（3008）以3075元開出，上漲30元；鴻海（2317）則以112元開出，上漲1.5元。
今航運股反彈，萬海、長榮漲幅逾3%，陽明漲幅逾1%。疫苗概念股則持續走低，高端跌逾7%，聯亞藥跌幅則逾5%。其他相關疫苗股也下跌，基亞跌幅逾1%。
美股道瓊工業指數上漲25.35點，或0.07%，收34823.35點；標普 500 指數上漲8.79點，或0.20%，收4367.48點；那斯達綜合指數上漲52.64點，或0.36%，收14684.6點；費城半導體指數下跌29.33點，或0.89%，收3259.42點。
----------------------------------------------------------------------

[72] 鎖定「科技+醫療」雙主流　富蘭克林高科技基金兩個月漲幅居冠

▲八檔一般科技/股票型基金績效。（圖／記者李道正製表）
記者李道正／綜合報導
美國企業公布亮眼財報，一掃變種病毒威脅的陰霾，再加上美債殖利率反彈、緩解債市避險氛圍，在景氣回溫的帶動下，激勵美股走勢，從五月十二日的波段低點以來，納斯達克上漲12%，演出解封後的慶祝行情。
對於台灣的投資人來說，想要投資美股，主要有兩種方法，一種是到海外開證券戶，直接買賣美股，零手續費但得自行承擔匯費及匯兌風險；另一種則是在國內券商複委託，單筆買賣成本較高，且美股開盤跟台灣人的作息有時差，投資人不見得能夠即時因應市場氣氛來操作。
因此，建議對美股有興趣的投資人，直接投資美股科技基金，因為美股進入財報密集公佈期，根據Factset（7/16）統計，史坦普500大企業第二季獲利預估將創下2009年第四季以來最大增幅，其中科技類股獲利預估成長達31.2%，表現持續穩健。
投資人只要每個月定期定額投入，再加上逢低單筆加碼買進，即可輕鬆成為亞馬遜（Amazon）、微軟（Microsoft）、臉書（Facebook）、輝達（NVIDIA）及Google母公司Alphabet的股東，由專業的投資團隊及經理人代為觀測市場及投資布局，投資人只需要定期投資，長期下來，即可在報酬率上看到豐收的

▲財政部統計目前為止，紓困4.0公股銀行核准的戶數和金額。（圖／記者吳佳穎攝）
記者吳佳穎／台北報導
財政部國庫署今（22）日表示，紓困4.0公股銀行積極辦理紓困貸款，截至昨（21）日止，共核准2萬8536戶、金額達新台幣2663億元，戶數及金額核准率分別是98%及89% 。
而據金融監督管理委員會資料，截至7月19日止，公股銀行辦理各部會企業紓困方案、銀行自辦（企業戶、個人戶）累計受理申請戶數31萬1069戶、金額為3.2兆元，核准戶數為30萬7336戶、金額達3.1兆元，核准戶數及金額占全體銀行比率，分別為67%及84%。
國庫署說，從6月4日受理紓困申請以來，每兩週追蹤，戶數或金額都呈倍數成長，成長率則是3位數，戶數成長率為370%、金額則有430%。6月4日至6月18日的戶數核准率為91%，至今已達98%，而金額核准率也從76%上升至89%。
至於外界關心哪家公股銀行是承辦冠軍，國庫署說，並沒有比賽、訂定審核天數及關鍵績效指標（KPI），只是鼓勵各銀行在防疫安全情況下，辦理紓困方案。
另外，財政部對於近日外界關注有關夜市攤商、補教業及旅宿業申請紓困貸款所遇問題一事表示，紓困貸款係配合各主管機關所訂紓困振興辦法辦理，銀行受理紓困案件後，仍需遵循相關金融法規及銀行公會會員徵信授信準則，依授信5P原則進行評估審查。
----------------------------------------------------------------------

[81] 烟花強灌雨　全台「4水庫放水了」！睽違422天發洩洪警戒

▲石門水庫放流中！全台「4水庫洩洪」。（圖／北水局）
記者許力方／綜合報導
中颱烟花來勢洶洶，中央氣象局預報，23日受颱風外圍環流影響，雨勢更快更急，中部以北、東北部地區以及南部山區易有豪大雨，尤其北部山區將出現局部豪雨以上等級降雨。翡翠水庫22日12時水位為159.56公尺，預計15時起進行每秒310立方公尺調節性放水；此外，苗栗明德水庫也因逼近滿水位，歷經大旱災後，睽違14個月再度洩洪，目前石門、寶二、明德水庫正在放流警戒。
氣象局針對台中以北9縣市持續發布「豪雨特報」，22日已有局部豪大雨發生，翡管局表示，氣象局預報翡翠水庫集水區將有豪雨等級降雨，為調節水庫水位、維護大壩安全，預計於22日15時起進行每秒310立方公尺調節性放水。未來並

▲財信傳媒董事長謝金河說，未來航空業需要的飛機可能和現在不太一樣，會產生新商機。（資料圖／記者湯興漢攝）
記者吳佳穎／台北報導
財信傳媒董事長謝金河今（22）日在臉書Po文指出，近期因台灣奧運選手搭經濟艙引熱議，令他聯想到解封後的「飛機座艙」勢必兼顧防疫和減碳考量，機票價格可能會大幅提升。加上最近聯合航空向波音訂了270架飛機，是今年最大宗的新機訂單，添購新機的效應，很可能像貨櫃航運業一樣，從破產中大翻身，國內波音、空巴恊力台廠的股價也可能有新機會。
謝金河指出，這次東京奧運，台灣奧運國手包機，因為小戴的「好懷念長榮航空，可以搭乘商務艙」，引發熱烈爭議，總統及行政院長都為此道歉。他無意加入這個論戰，倒是「商務艙」，「梅花座經濟艙」引發的一些聯想。
他說，這兩年疫情重創全球航空業，要找到一個比航空業還慘的慘業可能不多！現在大家都在想，全球都普遍打完疫苗，迎接疫後的解封，這個世界會發生什麼變化？而面對疫情帶來的撞擊，疫情後的新飛機，可能也會有不同的造型。一位業者告訴我，未來飛機座位不能坐太密，可能要改採梅花座，未來飛機票價會大幅提升。也就是新世代的飛機，會跟過去的飛機不一樣，除了要兼顧碳排放，還要考慮防疫。
從投資的角度來看，所有能夠大起的產業，通常都會經歷過一段「寒澈骨」的煎熬。像全球航商在2015年到2017年的倒閉潮，韓進破產是代表作。大家都記得，那時陽明快倒，後來私募，每股5元，7元，大家都不敢認，後來交通部，國發基金及高雄港務公司把額度吃下，如今暴賺，這回陽明詢價圈購一股182元，高檔增資風險升高。
「很多人喜歡，今天買到的股票，明天就漲停賣出。」謝金河表示，全世界沒有這麼好的事，如果從長缐的投資與布局的角度來看，現在可以開始注意「慘到極致」的航空業。
最近聯合航空向波音訂了270架飛機，這是今年最大宗的新機訂單，上一次聯合航空的訂機是2011年的460架，現在大約相隔十年。大家可以密切注意其他航空公司的新機需求，這個效應很可能像這次貨櫃航運業從破產中大翻身。
他說，這次貨櫃航運的新船必須符合碳中和的環保需求，必須加裝脫硫設備；飛機除了環保，還要有防疫的考量，這會是新世代的新舊機種大交替的新時代，也許會是下一個產業的大浪。
但趨勢的形成，通常不是瞬間發生，這個大浪可能一年，兩年，三年甚至更久，也許長線投資者可以開始留意。謝金河說：「國內很多波音，空巴恊力廠，股